# Install packages

In [1]:
%%capture
!pip install super-gradients
!git clone https://github.com/Deci-AI/super-gradients.git /content/super_gradients_folder
!sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /content/super_gradients_folder/src/super_gradients/training/pretrained_models.py
!sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /content/super_gradients_folder/src/super_gradients/training/utils/checkpoint_utils.py
!pip install -e /content/super_gradients_folder
!pip install onnxruntime

## Select model

In [7]:
model_name = 'yolonas_s' #@param ["yolonas_s", "yolonas_m", "yolonas_l"]
quantization = 'fp16' #@param ["fp16", "uint8"]
input_width = 640 #@param {type:"slider", min:320, max:640, step:320}
input_height = 640 #@param {type:"slider", min:320, max:640, step:320}

MODEL_FILENAME = f"{model_name}_{quantization}_{input_width}x{input_height}.onnx"

In [8]:
from super_gradients.common.object_names import Models
from super_gradients.training import models

MODEL_NAMES = {
    "yolonas_s": Models.YOLO_NAS_S,
    "yolonas_m": Models.YOLO_NAS_M,
    "yolonas_l": Models.YOLO_NAS_L,
}

model = models.get(MODEL_NAMES[model_name], pretrained_weights="coco")
model.eval()
model.prep_model_for_conversion(input_size=[1, 3, input_height, input_width])

WARNING: :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco dataset, published under the following terms: https://cocodataset.org/#termsofuse


W0314 10:29:37.193045 135582542787200 checkpoint_utils.py:1565] :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco dataset, published under the following terms: https://cocodataset.org/#termsofuse


In [9]:
from super_gradients.conversion import DetectionOutputFormatMode
from super_gradients.conversion.conversion_enums import ExportQuantizationMode

quantization_mode = ExportQuantizationMode.FP16 if quantization == "fp16" else ExportQuantizationMode.INT8
model.export(
  MODEL_FILENAME,
  input_image_shape=(input_height, input_width),
  num_pre_nms_predictions=300,
  max_predictions_per_image=5,
  nms_threshold=0.7,
  confidence_threshold=0.4,
  output_predictions_format=DetectionOutputFormatMode.FLAT_FORMAT,
  quantization_mode=quantization_mode
)

Model exported successfully to yolonas_s_fp16_640x640.onnx
Model expects input image of shape [1, 3, 640, 640]
Input image dtype is torch.uint8
Exported model already contains preprocessing (normalization) step, so you don't need to do it manually.
Preprocessing steps to be applied to input image are:
Sequential(
  (0): CastTensorTo(dtype=torch.float16)
  (1): ApplyMeanStd(mean=[0.], scale=[255.])
)

Exported model contains postprocessing (NMS) step with the following parameters:
    num_pre_nms_predictions=300
    max_predictions_per_image=5
    nms_threshold=0.7
    confidence_threshold=0.4
    output_predictions_format=DetectionOutputFormatMode.FLAT_FORMAT

Exported model is in ONNX format and can be used with ONNXRuntime
To run inference with ONNXRuntime, please use the following code snippet:

    import onnxruntime
    import numpy as np
    session = onnxruntime.InferenceSession("yolonas_s_fp16_640x640.onnx", providers=["CUDAExecutionProvider", "CPUExecutionProvider"])
    input

In [10]:
import onnxruntime as ort
import numpy as np

dummy_input = np.random.randint(0, 255, (1, 3, input_width, input_height)).astype(np.uint8)

ort_session = ort.InferenceSession(MODEL_FILENAME, providers=["ROCMExecutionProvider"])
ort_session.run(None, {"input": dummy_input})

[array([[0.0000000e+00, 5.0000000e-01, 0.0000000e+00, 6.3950000e+02,
         6.3900000e+02, 9.4970703e-01, 5.0000000e+01]], dtype=float32)]

In [11]:
from google.colab import files

files.download(MODEL_FILENAME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>